<a href="https://colab.research.google.com/github/TEJRAJ009/AI-ML-Based-Customer-Service-Tool-for-Automated-Warehouses-/blob/dev/rag_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
    datasets==2.14.5 \
    groq==0.8.0 \
    "semantic-router[local]==0.0.45" \
    pinecone-client==4.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv2-semantic-chunks", split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

Dataset({
    features: ['id', 'title', 'content', 'prechunk_id', 'postchunk_id', 'arxiv_id', 'references'],
    num_rows: 1000
})

In [ ]:
data[1]

{'id': '2401.04088#1',
 'title': 'Mixtral of Experts',
 'content': 'Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/mixtral-of-experts/ # Introduction In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights, licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As it only uses a subset of its parameters for every token, Mixtral allows faster inference speed at low batch-sizes, and higher throughput at large batch-sizes. Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the feedforward block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router network chooses two of these groups (the â',
 'prechunk_id': '2401.04088#0',
 'postchunk_id': '2401.04088#2',
 'arxiv_id': '2401.04088',
 'references': ['1905.07830']}

In [ ]:
data = data.map(lambda x:{
    "id" :x["id"],
    "metadata" : {
        "title" : x["title"],
        "content" : x["content"],
    }
})

#drop uneeded colums
data = data.remove_columns(["title", "content", "prechunk_id",
    "postchunk_id", "arxiv_id", "references"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['id', 'metadata'],
    num_rows: 1000
})

In [ ]:
from semantic_router.encoders import HuggingFaceEncoder
encoder = HuggingFaceEncoder(name = "dwzhu/e5-base-4k" )

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/225M [00:00<?, ?B/s]

In [ ]:
encoder.device

'cuda'

In [ ]:
#test embeding
embeds = encoder(["this is a test"])

In [ ]:
dims = len(embeds[0])
dims

768

In [ ]:
#creating vector DB
import os
from pinecone import Pinecone

#initialize connection to pinecone
api_key = "ca77b0e8-a969-4548-938e-e4865d9fb03d"
pc = Pinecone(api_key = api_key)

In [ ]:
from pinecone import ServerlessSpec
spec = ServerlessSpec( cloud = "aws", region= 'us-east-1')


Creating an index, we set dimension equal to the dimensionality of our encoder (384), and use a metric also compatible with the model (this can be cosine). We also pass our spec to index initialization


groq api = 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

In [ ]:
import time

index_name = "vectortest"
existing_indexs = [index_info["name"] for index_info in pc.list_indexes()]

#check if index already exists (it should't if this is first time)
if index_name not in existing_indexs:
  #if does not exist , create index
  pc.create_index(
      name = index_name,
      dimension = dims,
      metric = "cosine",
      spec= spec
  )

  #wait for index to initialized
  while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

#connect to the index
index = pc.Index(index_name)
time.sleep(1)

#view index stats
index.describe_index_stats()


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
from tqdm.auto import tqdm

In [ ]:
batch_size = 256

for i in tqdm(range(0, len(data), batch_size)):
    #find end of batch
    i_end = min(len(data),i+batch_size)

    #create batch
    batch = data[i:i_end]

    #create enbeding
    chunks = [f'{x["title"]}: {x["content"]}' for x in batch["metadata"]]
    embeds = encoder(chunks)
    assert len(embeds) == (i_end-i)

    #upsert batch to pinecone
    to_upsert = list(zip(batch["id"],embeds, batch["metadata"]))
    index.upsert(vectors = to_upsert)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def get_docs(query: str , top_k : int) -> list[str]:
  #encode qurey
  xq = encoder([query])

  #search pinecone index
  res = index.query(vector=xq, top_k = top_k , include_metadata =True)

  #get doc text
  docs = [x["metadata"]['content'] for x in res["matches"]]
  return docs



In [ ]:
query = "can you tell me about llama LLMs ?"
docs = get_docs(query, top_k = 5)
print("\n --- \n".join(docs))

â ¢ LLaMA-2-7B (Touvron et al., 2023b): it is an updated version of LLaMA (Touvron et al., 2023a). It has been pre-trained on a mixture of publicly available online data of 2T tokens. # 2.2 Results and Analysis We report the evaluation results of LLMs after train- ing with the benchmark leakage settings in Table 1 and Table 2. Overall, different levels of data leak- age result in inflated model performance on bench- marks. We have the following observations. evaluation into an in-domain test task, making it easier for LLMs to achieve higher results. An in- triguing finding occurs when we examine the result on the Chinese benchmark C3-Dialog. Despite the pre-training corpus of the four LLMs containing very little Chinese data, using training sets doubles their evaluation scores, e.g., elevating GPT-Neo- 1.3Bâ s score from 24.18 to 48.62. This observation underscores the significance of avoiding training set leakage in pre-training, as it can lead to spuri- ous performance improvements t

In [ ]:
from groq import Groq

api_key = "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"
groq_client = Groq(api_key = api_key)


In [ ]:
def generate (query : str , docs:list[str]):
  system_message = (
        "You are a helpful assistant that answers questions about AI using the "
        "context provided below.\n\n"
        "CONTEXT:\n"
        "\n---\n".join(docs)
    )
  messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query}
    ]
    # generate response
  chat_response = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages
    )
  return chat_response.choices[0].message.content

In [ ]:
query = "can you tell me about llama LLMs ?"
docs = get_docs(query, top_k = 5)
out = generate(query= query , docs = docs)
print(out)

LLaMA is a series of large language models (LLMs) that have been trained on a massive amount of text data. Here are some key points about LLaMA LLMs:

1. **LLaMA-2-7B**: This is an updated version of the original LLaMA model. It has been pre-trained on a mixture of publicly available online data of 2 trillion tokens.

2. **LLaMA-65B**: This is a larger version of the LLaMA model, with 65 billion parameters.

3. **OpenLLaMA-3B**: This is an open-source project that aims to reproduce the LLaMA model with a permissive license. It has been pre-trained on the RedPajama dataset of over 1.2 trillion tokens.

4. **Capabilities**: LLaMA LLMs have demonstrated strong performance on a range of natural language processing tasks, including language translation, question answering, and text generation.

5. **Training data**: The training data for LLaMA LLMs includes a massive amount of text data from the internet, which can lead to issues with data leakage and overspecialization to specific tasks.



* each query cost 13 run units
* i get 1 million every month
* which comes 10,00,000 / 13 = 76923 querys
* aka penty enough